In [6]:
def deco(func):
  def inner():
    print("running inner()")
  return inner
@deco
def target():
  print("running target()")
target()

running inner()


In [7]:
target()

running inner()


In [8]:
target

<function __main__.deco.<locals>.inner()>

In [10]:
def logger(func):
    def wrapper(*args, **kwargs):
        print(f"[LOG] {func.__name__} called with args={args}, kwargs={kwargs}")
        result = func(*args, **kwargs)
        print(f"[LOG] {func.__name__} returned {result}")
        return result
    return wrapper


@logger
def add(a, b):
    return a + b
add(5,12)

[LOG] add called with args=(5, 12), kwargs={}
[LOG] add returned 17


17

In [11]:
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func


@register
def f1():
    print('running f1()')


@register
def f2():
    print('running f2()')


def f3():
    print('running f3()')


def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()


if __name__ == '__main__':
    main()

running register(<function f1 at 0x79dc46dc2fc0>)
running register(<function f2 at 0x79dc46dc2660>)
running main()
registry -> [<function f1 at 0x79dc46dc2fc0>, <function f2 at 0x79dc46dc2660>]
running f1()
running f2()
running f3()


In [12]:
promos = []

def promotion(promo_func):
    """promotion() decorator는 promo_func를 promos 리스트에 추가한 후 그대로 반환"""
    promos.append(promo_func)
    return promo_func


@promotion
def fidelity_promo(order):
    """충성도 점수가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0


@promotion
def bulk_item_promo(order):
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount


@promotion
def large_order_promo(order):
    """10종류 이상의 상품을 구입하면 전체 7% 할인 적용"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0


def best_promo(order):
    """최대로 할인받은 금액을 반환한다."""
    return max(promo(order) for promo in promos)

In [13]:
def f1(a):
  print(a)
  print(b)
f1(3)

3


NameError: name 'b' is not defined

In [14]:
b = 6
def f2(a):
  print(a)
  print(b)
  b = 9
f2(3)

3


UnboundLocalError: cannot access local variable 'b' where it is not associated with a value

In [15]:
b = 6
def f3(a):
  global b
  print(a)
  print(b)
  b = 9

f3(3)

3
6


In [16]:
b

9

In [17]:
class Averager():
  def __init__(self):
    self.series = []
  def __call__(self,new_value):
    self.series.append(new_value)
    total = sum(self.series)
    return total/len(self.series)

In [18]:
avg = Averager()
avg(10)

10.0

In [19]:
avg(11)

10.5

In [20]:
avg(12)

11.0

In [23]:
def make_averager():
  series = []

  def averager(new_value):
    series.append(new_value)
    total = sum(series)
    return total/len(series)
  return averager

In [24]:
avg = make_averager()
avg(10)

10.0

In [25]:
avg(11)

10.5

In [26]:
avg(12)

11.0

In [27]:
avg.__code__.co_varnames

('new_value', 'total')

In [28]:
avg.__code__.co_freevars

('series',)

In [29]:
avg.__closure__

(<cell at 0x79dc2e9cd9f0: list object at 0x79dc2e76e6c0>,)

In [30]:
avg.__closure__[0].cell_contents

[10, 11, 12]

In [32]:
def make_averager():
  count = 0
  total = 0

  def averager(new_value):
    count += 1
    total += new_value
    return total / count
  return averager

In [33]:
agv = make_averager()
avg(10)

10.75

In [34]:
def make_averager():
  count = 0
  total = 0

  def averager(new_value):
    nonlocal count, total
    count += 1
    total += new_value
    return total / count
  return averager

In [35]:
avg = make_averager()
avg(10)

10.0

In [36]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [37]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__=='__main__':
    print('*'*40, 'Calling snooze(.123)')
    snooze(.123)
    print('*'*40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12315966s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000095s] factorial(1) -> 1
[0.00001366s] factorial(2) -> 2
[0.00001980s] factorial(3) -> 6
[0.00002584s] factorial(4) -> 24
[0.00003173s] factorial(5) -> 120
[0.00004055s] factorial(6) -> 720
6! = 720


In [38]:
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        arg_lst = []
        name = func.__name__
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k,w) for k,w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [39]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

if __name__=='__main__':
    print('*'*40, 'Calling snooze(.123)')
    snooze(.123)
    print('*'*40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12318420s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000095s] factorial(1) -> 1
[0.00002241s] factorial(2) -> 2
[0.00003338s] factorial(3) -> 6
[0.00004458s] factorial(4) -> 24
[0.00005460s] factorial(5) -> 120
[0.00006938s] factorial(6) -> 720
6! = 720


In [40]:
factorial.__name__

'factorial'